In [3]:
%load_ext zipline

In [4]:
from zipline.api import *
from zipline.algorithm import *
import zipline
from plotly.offline import init_notebook_mode , plot , iplot , download_plotlyjs
import pandas as pd
import numpy as np
import plotly as pyfolio
import cufflinks as cf
from datetime import datetime
import pyfolio as pf
import matplotlib.pyplot as plt
from pandas.plotting import _converter
init_notebook_mode(connected=True)
cf.go_offline()

In [9]:
def initialize(context):
    '''初始化證券池'''
#     set_benchmark(symbol('AAPL'))
    context.security = [symbol('AAPL'),symbol('AMZN'),symbol('GOOGL'),symbol('EBAY'),symbol('ACTG')]
    schedule_function(ma_crossover_handling,
                      date_rules.every_day(),
                      time_rules.market_open(hours = 1)
                     )
    
def ma_crossover_handling(context , data):
    days = 26
    shr = 12
    
    for st in context.security :
        h = data.history(st ,
                            'price' , days ,
                            '1d' )
        dea = h.ewm(span=days, adjust=False).mean()
        
        dif = h[-shr:].ewm(span=shr, adjust=False).mean()
        
        macd = dif - dea
        
#         dea = last day dea + 0.2*(today dif - last day eda)
        ct = 0 
        open_orders = get_open_orders()
        '''
        other alg
        dif = ema(12) - ema(26)
        dea = last day dea + 0.2*(today dif - last day eda)
        macd = dif - dea
        '''
        if dea[-1] > 0 and dif[-1] > 0 and round(macd[-1],1) !=0 :
            if dif[-1] > dea[-1] : 
                if st not in open_orders:
                    order_target_percent(st , 0.2 )
#                     print("buy trade at {}".format(context.get_datetime()))
                    ct += 1
        elif dea[-1] < 0 and dif[-1] < 0 and round(macd[-1],1) !=0.2 :
            if dif[-1] < dea[-1] : 
                if st in open_orders:
                    order_target_percent(st , -0.2 )
#                     print("sell trade at {}".format(context.get_datetime()))
                    ct -= 1 
        
    record(macd = macd[-1] , ct = ct , dea = dea[-1] , dif = dif[-1])
    
def analyze(context , pref ):
    print('initialize cash %s '%str(pref.starting_cash[1]))
    print('results cash %s '%str(pref.starting_cash[-1]))
    print('max_drawdown ' + str(pref.max_drawdown.mean()) + '%' )
    print('portfolio_value %s'%pref.portfolio_value[-1])
    print('beta -> %s'%pref.beta.dropna().mean())
    print('sharpe -> %s'%pref.sharpe.mean())
    print('max drawdown -> ' + str( pref.max_drawdown.mean() ) + '%') 
    print('alpha -> %s '%pref.alpha.mean())
    layout = cf.Layout(
        height = 300
    )
    layoutf = cf.Layout(
        height = 500
    )
#     returns , positions , transactions = pf.utils.extract_rets_pos_txn_from_zipline(pref)
#     pf.create_returns_tear_sheet(returns , benchmark_rets = None)
    pref[['macd']].iplot(hline=[0],title = 'macd plot',layout = layout)
    pref['ct'].iplot(title = 'macd trading plot',layout = layout)
    pref[['dea','dif']].iplot(title = 'cross',layout=layoutf)    
    pref[['benchmark_period_return','algorithm_period_return']].iplot()
    pref['gross_leverage'].iplot(title='gross lev')
    pref['returns'].iplot(title='return')

In [10]:
%zipline --bundle quantopian-quandl --start 2016-1-4 --end 2016-2-4 -o backtest.pickle

initialize cash 10000000.0 
results cash 6008734.285694999 
max_drawdown -0.025983652913282335%
portfolio_value 9609113.205695
beta -> nan
sharpe -> -4.76974825120381
max drawdown -> -0.025983652913282335%
alpha -> nan 


,algo_volatility,algorithm_period_return,alpha,benchmark_period_return,benchmark_volatility,beta,capital_used,ct,dea,dif,...,short_exposure,short_value,shorts_count,sortino,starting_cash,starting_exposure,starting_value,trading_days,transactions,treasury_period_return
2016-01-04 21:00:00+00:00,NaN,0.000000,None,-0.013979,NaN,None,0.000000e+00,0,4.941951,4.483659,...,0.0,0.0,0,NaN,1.000000e+07,0.00,0.00,1,[],0.0
2016-01-05 21:00:00+00:00,0.002248,-0.000200,None,-0.027763,0.0,None,-3.996578e+06,0,4.869696,4.387704,...,0.0,0.0,0,-11.224972,1.000000e+07,0.00,0.00,2,"[{'dt': 2016-01-05 21:00:00+00:00, 'amount': 2...",0.0
2016-01-06 21:00:00+00:00,0.007838,-0.001138,None,-0.041355,0.0,None,5.312645e+03,0,4.848351,4.206224,...,0.0,0.0,0,-10.877763,6.003422e+06,3994575.30,3994575.30,3,"[{'dt': 2016-01-06 21:00:00+00:00, 'amount': -...",0.0
2016-01-07 21:00:00+00:00,0.097085,-0.013708,None,-0.054756,0.0,None,0.000000e+00,0,4.777938,4.135072,...,0.0,0.0,0,-8.629932,6.008734e+06,3979888.93,3979888.93,4,[],0.0
2016-01-08 21:00:00+00:00,0.084142,-0.016637,None,-0.067970,0.0,None,0.000000e+00,0,4.717296,4.024829,...,0.0,0.0,0,-9.139727,6.008734e+06,3854189.66,3854189.66,5,[],0.0
2016-01-11 21:00:00+00:00,0.088989,-0.012714,None,-0.080999,0.0,None,0.000000e+00,0,4.621013,3.972941,...,0.0,0.0,0,-6.349406,6.008734e+06,3824899.61,3824899.61,6,[],0.0
2016-01-12 21:00:00+00:00,0.087521,-0.009445,None,-0.093847,0.0,None,0.000000e+00,0,4.546605,3.920571,...,0.0,0.0,0,-4.346043,6.008734e+06,3864127.68,3864127.68,7,[],0.0
2016-01-13 21:00:00+00:00,0.124850,-0.027537,None,-0.106514,0.0,None,0.000000e+00,0,4.479154,3.862412,...,0.0,0.0,0,-6.929840,6.008734e+06,3896819.55,3896819.55,8,[],0.0
2016-01-14 21:00:00+00:00,0.128791,-0.020921,None,-0.119005,0.0,None,0.000000e+00,0,4.442366,3.873016,...,0.0,0.0,0,-4.926197,6.008734e+06,3715892.41,3715892.41,9,[],0.0
2016-01-15 21:00:00+00:00,0.132583,-0.033572,None,-0.131320,0.0,None,0.000000e+00,0,4.355410,3.837742,...,0.0,0.0,0,-6.556573,6.008734e+06,3782057.14,3782057.14,10,[],0.0


limit_output extension: Maximum message size of 10000 exceeded with 15035 characters

In [18]:
i = 0
if i == round(0.022222,1) :
    print('HK')

HK
